In [12]:
import os, sys                               # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

# updating nipype from 1.7.0 -> 1.8.1, will hopefully ifx no PE estimates founds error # didn't fix

### Copes at level 2:
##### Model 0:
1. FS
2. SS
3. Go
4. FS-Go
5. FS-SS
6. SS-Go

##### Model 1:
1. response_left
2. response_right
3. response_left-response_right

NOTE: had to manually change reg expression for this dataset as fwhm name was longer than all others

In [13]:
# dataset = 'Leipzig_7T_SM'
# dataset = 'Leipzig_7T_GdH'
dataset = 'aron_3T'

#### !!! must manually change subject numbers in nipype code based on dataset !!!

In [3]:
# general set-up
project_folder = '/home/scotti/projects/3t_7t_sst_comparison'

if dataset == 'Leipzig_7T_SM':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['1p6']
    
elif dataset == 'Leipzig_7T_GdH':
    task = 'stop'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['1p5']
    
elif dataset == 'aron_3T':
    task = 'stopsignal'
    spaces = ['T1w'] 
    model_ns = ['0']
    fwhms = ['5p0']#['3p125']
    
if model_ns[0] == '0':
    contrasts = ['0','1','2','3','4','5']  # task from second level model
elif model_ns[0] == '1':
    contrasts = ['0','1','2'] # motor from second level model

subject_ids = [x.split('/')[-5].split('-')[-1] for x in sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 
                                     'subject_level_model', dataset, 'sub-*', 'func',f'fwhm-{fwhms[0]}',f'model-{model_ns[0]}',
                                     f'*task-{task}_space-MNI*contrast-0_desc-zstat*')))]
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders', dataset)
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat_hp_sct', 'subject_level_model', dataset)

template_brain = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
template_brain_mask = os.path.join(project_folder, 'sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

output_dir = os.path.join(project_folder, 'derivatives', "glm_feat_hp_sct", "group_level_model", dataset,f'model-{model_ns[0]}')

templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stopsignal_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stopsignal_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*task-stopsignal_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}


In [4]:
print(f"""
dataset : {dataset}
fwhm : {fwhms}
model_n : {model_ns}
task : {task}
no. subs : {len(subject_ids)}
subs : {subject_ids}
work_dir : {work_dir}
slm folder : {slm_folder}
output dir : {output_dir}
contrasts : {contrasts}

## if uneven number of template / designs / subs the code will crash
no.copes = {len(glob.glob(templates['level2_cope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.varvopes = {len(glob.glob(templates['level2_varcope'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
no.tdofs = {len(glob.glob(templates['level2_tdof'].format(fwhm=fwhms[0],model_n=model_ns[0],contrast_n=0)))}
""")


dataset : aron_3T
fwhm : ['5p0']
model_n : ['0']
task : stopsignal
no. subs : 14
subs : ['01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12', '13', '14', '15']
work_dir : /home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T
slm folder : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/subject_level_model/aron_3T
output dir : /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/aron_3T/model-0
contrasts : ['0', '1', '2', '3', '4', '5']

## if uneven number of template / designs / subs the code will crash
no.copes = 14
no.varvopes = 14
no.tdofs = 14



In [5]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [6]:
##
workflow = pe.Workflow(name='feat_level3_sst_hp')
workflow.base_dir = work_dir
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [7]:
def get_subject_ids():
#     if dataset == 'Leipzig_7T_SM':
    subject_ids = ['01', '02', '03', '04', '05', '06', '07', '09', '10', '11', '12', '13', '14', '15']
        
#     elif dataset == 'Leipzig_7T_GdH':
#     subject_ids = ['BI3T', 'DA9T', 'FMFT', 'GAIT', 'HCBT', 'KCAT', 'KP6T', 'LV2T', 'MRCT', 'NM3T', 'PF5T', 
#                        'RSIT', 'SC1T', 'SPGT', 'TS6T', 'UM2T', 'VL1T', 'WSFT', 'WW2T', 'ZK4T']
        
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids): #=subject_ids):
    print(second_level_contrast)
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    # contrasts (3rd-level)
    third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                             ('-Group mean', 'T', ['intercept'], [-1.0])
                             ]
    
    return third_level_contrasts, regressors


contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')



In [8]:
# flameo = pe.Node(
#     interface=fsl.FLAMEO(run_mode='flame1'),
#     name="flame1")

# flameo.inputs.mask_file = template_brain_mask

# workflow.connect([
#     (copemerge, flameo, [('merged_file', 'cope_file')]),
#     (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
#     (level3model, flameo, [('design_mat', 'design_file'),
#                            ('design_con', 't_con_file'), 
#                            ('design_grp', 'cov_split_file')]),
# ])

flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False   # run with automatic outlier detection

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [9]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [10]:
# ## Datasink
# datasink = pe.Node(nio.DataSink(), name='sinker')
# datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-sstmsit","task-sst")

# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

# ## todo: substitutions
# workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
# workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
# workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
# workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

# ## cluster results
# workflow.connect(grf_cluster, 'threshold_file', datasink, 'grf_thresholded_zstats_file')
# workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'grf_localmax_txt_file')
# workflow.connect(grf_cluster, 'index_file', datasink, 'cluster_indices')

In [10]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=output_dir

if len(fwhms[0]) == 3:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]
elif len(fwhms[0]) == 5:
    substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                             'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                            (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{5})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                              'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                            # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                            #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                           ]    

datasink.inputs.regexp_substitutions = substitutions_regexp

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 100})

220923-15:35:14,621 nipype.workflow INFO:
	 Workflow feat_level3_sst_hp settings: ['check', 'execution', 'logging', 'monitoring']
220923-15:35:14,918 nipype.workflow INFO:
	 Running in parallel.
220923-15:35:14,937 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 100.00/100.00, Free processors: 10/10.
220923-15:35:15,2 nipype.workflow INFO:
	 [Job 0] Cached (feat_level3_sst_hp.subject_id_getter).
220923-15:35:15,4 nipype.workflow INFO:
	 [Job 1] Cached (feat_level3_sst_hp.get_volume).
220923-15:35:15,48 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_0_fwhm_5p0_model_n_0/selector".
220923-15:35:15,50 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.selector" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_1_fwhm_5p0_mod

220923-15:35:18,938 nipype.workflow INFO:
	 [Job 8] Completed (feat_level3_sst_hp.contrastgen_l3).
220923-15:35:18,940 nipype.workflow INFO:
	 [Job 9] Completed (feat_level3_sst_hp.contrastgen_l3).
220923-15:35:18,941 nipype.workflow INFO:
	 [Job 10] Completed (feat_level3_sst_hp.contrastgen_l3).
220923-15:35:18,943 nipype.workflow INFO:
	 [Job 11] Completed (feat_level3_sst_hp.contrastgen_l3).
220923-15:35:18,944 nipype.workflow INFO:
	 [Job 12] Completed (feat_level3_sst_hp.contrastgen_l3).
220923-15:35:18,945 nipype.workflow INFO:
	 [Job 13] Completed (feat_level3_sst_hp.contrastgen_l3).
220923-15:35:18,947 nipype.workflow INFO:
	 [MultiProc] Running 4 tasks, and 14 jobs ready. Free memory (GB): 99.20/100.00, Free processors: 6/10.
                     Currently running:
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.cop

220923-15:35:38,969 nipype.workflow INFO:
	 [MultiProc] Running 2 tasks, and 8 jobs ready. Free memory (GB): 99.60/100.00, Free processors: 8/10.
                     Currently running:
                       * feat_level3_sst_hp.varcopemerge
                       * feat_level3_sst_hp.copemerge
220923-15:35:39,42 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.l3model" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_2_fwhm_5p0_model_n_0/l3model".
220923-15:35:39,45 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.l3model" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_5_fwhm_5p0_model_n_0/l3model".
220923-15:35:39,44 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.l3model" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_3_fwhm_5

220923-15:35:57,131 nipype.workflow INFO:
	 [Node] Executing "flameo" <nipype.interfaces.fsl.model.FLAMEO>
220923-15:35:58,978 nipype.workflow INFO:
	 [MultiProc] Running 10 tasks, and 2 jobs ready. Free memory (GB): 98.00/100.00, Free processors: 0/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
220923-15:40:10,361 nipype.workflow INFO:
	 [Node] Finished "flameo", elapsed time 271.217511s.
220923-15:40:11,273 nipype.workflow INFO:
	 [Job 34] Completed (feat_level3_sst_hp.flameo).
220923-1

220923-15:40:19,426 nipype.workflow INFO:
	 [Node] Cached "_smoothestimate1" - collecting precomputed outputs
220923-15:40:19,445 nipype.workflow INFO:
	 [Node] "_smoothestimate1" found cached.
220923-15:40:20,435 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate1", elapsed time 1.039335s.
220923-15:40:21,281 nipype.workflow INFO:
	 [Job 46] Completed (feat_level3_sst_hp.smoothestimate).
220923-15:40:21,283 nipype.workflow INFO:
	 [Job 107] Completed (_smoothestimate1).
220923-15:40:21,285 nipype.workflow INFO:
	 [MultiProc] Running 8 tasks, and 7 jobs ready. Free memory (GB): 98.40/100.00, Free processors: 2/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * f

...

220923-15:41:07,406 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_4_fwhm_5p0_model_n_0/_run_mode_flame1/_threshold_2.3/grf_cluster/mapflow/_grf_cluster0".
220923-15:41:07,401 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_4_fwhm_5p0_model_n_0/_run_mode_flame1/_threshold_3.1/grf_cluster/mapflow/_grf_cluster1".
220923-15:41:07,428 nipype.workflow INFO:
	 [Node] Executing "_grf_cluster1" <nipype.interfaces.fsl.model.Cluster>
220923-15:41:07,457 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster0" - collecting precomputed outputs
220923-15:41:07,460 nipype.workflow INFO:
	 [Node] "_grf_cluster0" found cached.
220923-15:41:07,462 nipype.workflow INFO:
	 [Node] Setting-up "_grf_cluster1" in "/home/scotti/projects/3t_7t_sst_compari

220923-15:41:09,507 nipype.workflow INFO:
	 [Node] Finished "sinker", elapsed time 0.046722s.
220923-15:41:11,329 nipype.workflow INFO:
	 [Job 73] Completed (feat_level3_sst_hp.grf_cluster).
220923-15:41:11,331 nipype.workflow INFO:
	 [Job 96] Completed (feat_level3_sst_hp.sinker).
220923-15:41:11,334 nipype.workflow INFO:
	 [MultiProc] Running 7 tasks, and 1 jobs ready. Free memory (GB): 98.60/100.00, Free processors: 3/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
220923-15:41:11,409 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_sst_hp.sinker" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/ar

220923-15:44:37,634 nipype.workflow INFO:
	 [Node] Setting-up "_smoothestimate0" in "/home/scotti/projects/3t_7t_sst_comparison/processing/nipype_workflow_folders/aron_3T/feat_level3_sst_hp/_contrast_n_5_fwhm_5p0_model_n_0/_run_mode_flame1/smoothestimate/mapflow/_smoothestimate0".
220923-15:44:37,651 nipype.workflow INFO:
	 [Node] Executing "_smoothestimate0" <nipype.interfaces.fsl.model.SmoothEstimate>
220923-15:44:38,583 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate1", elapsed time 0.929516s.
220923-15:44:38,712 nipype.workflow INFO:
	 [Node] Finished "_smoothestimate0", elapsed time 1.0488439999999999s.
220923-15:44:39,557 nipype.workflow INFO:
	 [Job 134] Completed (_smoothestimate0).
220923-15:44:39,559 nipype.workflow INFO:
	 [Job 135] Completed (_smoothestimate1).
220923-15:44:39,561 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 1 jobs ready. Free memory (GB): 98.80/100.00, Free processors: 4/10.
                     Currently running:
                     

220923-15:44:47,692 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster1" - collecting precomputed outputs
220923-15:44:47,696 nipype.workflow INFO:
	 [Node] "_grf_cluster1" found cached.
220923-15:44:47,684 nipype.workflow INFO:
	 [Node] Cached "_grf_cluster1" - collecting precomputed outputs
220923-15:44:47,706 nipype.workflow INFO:
	 [Node] "_grf_cluster1" found cached.
220923-15:44:49,567 nipype.workflow INFO:
	 [Job 76] Completed (feat_level3_sst_hp.grf_cluster).
220923-15:44:49,569 nipype.workflow INFO:
	 [Job 77] Completed (feat_level3_sst_hp.grf_cluster).
220923-15:44:49,571 nipype.workflow INFO:
	 [MultiProc] Running 6 tasks, and 2 jobs ready. Free memory (GB): 98.80/100.00, Free processors: 4/10.
                     Currently running:
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst_hp.flameo
                       * feat_level3_sst

220923-15:44:49,682 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/aron_3T/model-0/third_level_model/level3_tdof_ts/_contrast_n_5_fwhm_5p0_model_n_0/_run_mode_flame1/tdof_t2.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/aron_3T/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-5_grouplevelcontrast-2_flame-1_desc-tdof_t.nii.gz
220923-15:44:49,684 nipype.interface INFO:
	 sub: /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/aron_3T/model-0/third_level_model/grf_thresholded_zstats_file/_contrast_n_5_fwhm_5p0_model_n_0/_run_mode_flame1/_threshold_3.1/_grf_cluster0/zstat1_threshold.nii.gz -> /home/scotti/projects/3t_7t_sst_comparison/derivatives/glm_feat_hp_sct/group_level_model/aron_3T/model-0/model-0/model-0_fwhm-5p0_subjectlevelcontrast-5_grouplevelcontrast-1_flame-1_desc-zstat_voxelthreshold-3.1.nii.gz
220923-15:44:49,6